# Tiền xử lý dữ liệu (Data Preprocessing)

---

## Tóm tắt Pipeline

**Quy trình hoàn chỉnh:**

1. **Load & Clean Data** → Loại bỏ missing values, duplicates
2. **Detect Outliers (IQR)** → Phát hiện và có thể remove/cap
3. **Feature Engineering:**
   - Ordinal Encoding cho ordered categoricals
   - One-Hot Encoding cho nominal categoricals
   - Create interaction features nếu cần
4. **Remove Multicollinearity** → Loại features có correlation > 0.9
5. **Train-Test Split** → 80/20
6. **Standardization (Z-score):**
   - Fit trên train set
   - Transform cả train và test
7. **Handle Imbalance** → Oversample minority class (chỉ trên train set)
8. **Save Processed Data** → Lưu train.csv và test.csv


---

## Quy trình xử lý dữ liệu - Lý thuyết và Công thức

### 1. Outlier Detection - Phát hiện ngoại lai

**Phương pháp IQR (Interquartile Range):**

IQR được tính bằng hiệu của quartile thứ 3 và quartile thứ 1:

$$
\text{IQR} = Q_3 - Q_1
$$

Định nghĩa outliers:
$$
\text{Outlier} = \begin{cases}
x < Q_1 - 1.5 \times \text{IQR} & \text{(Lower bound)} \\
x > Q_3 + 1.5 \times \text{IQR} & \text{(Upper bound)}
\end{cases}
$$

**Implementation với NumPy:**
```python
Q1 = np.percentile(data, 25)
Q3 = np.percentile(data, 75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outlier_mask = (data < lower_bound) | (data > upper_bound)
```

---

### 2. Min-Max Normalization - Chuẩn hóa về [0,1]

**Công thức:**

$$
x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}
$$

Đưa dữ liệu về khoảng [0, 1], giữ nguyên phân phối tương đối.

**Implementation với NumPy:**
```python
x_min = data.min()
x_max = data.max()
x_normalized = (data - x_min) / (x_max - x_min)
```

**Ưu điểm:**
- Đơn giản, dễ hiểu
- Phù hợp với neural networks, algorithms yêu cầu bounded inputs

**Nhược điểm:**
- Nhạy cảm với outliers
- Không đảm bảo phân phối chuẩn

---

### 3. Log Transformation - Biến đổi logarit

**Công thức:**

$$
x_{log} = \log(x + 1)
$$

(Thêm 1 để tránh log(0) khi có giá trị 0)

**Implementation với NumPy:**
```python
x_log = np.log1p(data)  # log1p = log(1 + x)
```

**Mục đích:**
- Giảm skewness (độ lệch) của phân phối
- Xử lý phân phối lệch phải (right-skewed)
- Giảm ảnh hưởng của outliers

---

### 4. Decimal Scaling - Chuẩn hóa thập phân

**Công thức:**

$$
x_{scaled} = \frac{x}{10^j}
$$

trong đó $j$ là số chữ số của giá trị lớn nhất:
$$
j = \lceil \log_{10}(\max|x|) \rceil
$$

**Implementation với NumPy:**
```python
max_abs = np.abs(data).max()
j = np.ceil(np.log10(max_abs)) if max_abs > 0 else 0
x_scaled = data / (10 ** j)
```

---

### 5. Z-Score Standardization - Chuẩn hóa theo độ lệch chuẩn

**Công thức:**

$$
z = \frac{x - \mu}{\sigma}
$$

trong đó:
- $\mu$ = mean (trung bình)
- $\sigma$ = standard deviation (độ lệch chuẩn)

Kết quả có phân phối với mean = 0, std = 1.

**Implementation với NumPy:**
```python
mean = data.mean()
std = data.std()
z_scores = (data - mean) / std
```

**Ưu điểm:**
- Không bị ảnh hưởng bởi scale gốc của features
- Phù hợp với algorithms dựa trên gradient descent (Logistic Regression, Neural Networks)
- Cho các features có tầm ảnh hưởng như nhau

**Lưu ý:**
- Giả định phân phối chuẩn hoặc gần chuẩn
- Nhạy cảm với outliers (vì sử dụng mean và std)

---

### 6. Feature Encoding

#### 6.1. Ordinal Encoding - Mã hóa thứ tự

Cho các biến có thứ tự tự nhiên (ví dụ: Education Level):

$$
x_{encoded} \in \{0, 1, 2, ..., n-1\}
$$

**Implementation với NumPy:**
```python
mapping = {'Low': 0, 'Medium': 1, 'High': 2}
encoded = np.array([mapping.get(v, 0) for v in categorical_data])
```

**Khi nào dùng:**
- Biến có thứ tự tự nhiên (education, income level)
- Muốn giữ thông tin về ranking

---

#### 6.2. One-Hot Encoding - Mã hóa nhị phân

Biến đổi biến categorical thành nhiều biến nhị phân:

$$
x_{onehot} = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**Implementation với NumPy:**
```python
unique_values = np.unique(categorical_data)
for value in unique_values:
    one_hot = (categorical_data == value).astype(float)
    # Thêm vào feature matrix
```

**Khi nào dùng:**
- Biến nominal (không có thứ tự): Gender, Marital Status, Color
- Tránh model học được thứ tự không tồn tại

**Nhược điểm:**
- Tăng dimensionality (curse of dimensionality)
- Có thể gây multicollinearity

---

### 7. Multicollinearity Detection - Phát hiện đa cộng tuyến

**Correlation Coefficient (Pearson):**

$$
r_{xy} = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n}(x_i - \bar{x})^2} \sqrt{\sum_{i=1}^{n}(y_i - \bar{y})^2}}
$$

với $r_{xy} \in [-1, 1]$

**Quy tắc phát hiện:**
- $|r| > 0.9$ → Highly correlated → Loại bỏ một trong hai features

**Implementation với NumPy:**
```python
# Tính correlation matrix
corr_matrix = np.corrcoef(numeric_features, rowvar=False)

# Tìm các cặp highly correlated
for i in range(len(features)):
    for j in range(i+1, len(features)):
        if abs(corr_matrix[i, j]) > 0.9:
            # Drop feature j
            pass
```

**Tại sao cần loại bỏ multicollinearity:**
- Coefficients không ổn định
- Khó interpret model
- Có thể gây overfitting

---

### 8. Handling Class Imbalance - Xử lý mất cân bằng

#### 8.1. Random Oversampling

**Phương pháp:**
- Duplicate ngẫu nhiên các samples từ minority class cho đến khi cân bằng với majority class

$$
n_{minority, new} = n_{majority}
$$

**Implementation với NumPy:**
```python
# Tìm minority và majority class
class_counts = np.bincount(y)
minority_idx = np.where(y == minority_class)[0]

# Sample with replacement
n_samples_needed = n_majority - n_minority
oversampled_idx = rng.choice(minority_idx, size=n_samples_needed, replace=True)

# Combine
X_balanced = np.vstack([X, X[oversampled_idx]])
y_balanced = np.concatenate([y, y[oversampled_idx]])
```

**Ưu điểm:**
- Đơn giản, dễ implement
- Không mất thông tin

**Nhược điểm:**
- Có thể gây overfitting (duplicate exact samples)
- Không thêm thông tin mới

**Phương pháp nâng cao khác:**
- SMOTE (Synthetic Minority Over-sampling Technique): Tạo synthetic samples
- ADASYN: Adaptive Synthetic Sampling
- Class weights: Điều chỉnh loss function

---

### 9. Train-Test Split - Chia tập dữ liệu

**Tỷ lệ thông thường:** 80% Train / 20% Test

**Implementation với NumPy:**
```python
# Shuffle indices
indices = np.arange(len(data))
rng.shuffle(indices)

# Split
split_point = int(0.8 * len(data))
train_idx = indices[:split_point]
test_idx = indices[split_point:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
```

**Nguyên tắc quan trọng:**
- Shuffle trước khi chia (tránh bias)
- Không touch test set cho đến khi evaluate cuối cùng
- Standardization parameters (mean, std) phải fit trên train, áp dụng cho test

### Mục đích
Thiết lập môi trường làm việc, thêm `repo_root` để import các module trong `src`, import các hàm tiền xử lý và tải dữ liệu gốc `raw.csv` để kiểm tra kích thước ban đầu.

In [1]:
import os
import sys
import numpy as np

# Thiết lập đường dẫn để import module src
repo_root = os.getcwd()
if os.path.basename(repo_root) == 'notebooks':
    repo_root = os.path.dirname(repo_root)
if repo_root not in sys.path:
    sys.path.append(repo_root)

from src.data_processing import (
    load_raw_data,
    clean_strings,
    drop_leakage_and_id,
    split_numeric_categorical,
    detect_outliers_iqr,
    remove_outliers_iqr,
    min_max_normalize,
    log_transform,
    decimal_scaling,
    standardize_zscore,
)

raw_data = load_raw_data(os.path.join(repo_root, 'data/raw', 'raw.csv'))
print('Kích thước dữ liệu gốc:', raw_data.shape)



Kích thước dữ liệu gốc: (10127,)


In [2]:
from src.data_processing import preprocess_full

csv_path = os.path.join(repo_root, 'data/raw', 'raw.csv')
data, numeric_cols, cat_cols = preprocess_full(csv_path)
print('Kích thước sau clean + drop cột:', data.shape)
print('Cột số:', numeric_cols)
print('Cột phân loại:', cat_cols)


Kích thước sau clean + drop cột: (10127,)
Cột số: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
Cột phân loại: ['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']


In [3]:
# Kiểm tra nhanh giá trị Unknown / rỗng ở cột phân loại
for name in cat_cols:
    col = data[name]
    num_unknown = np.count_nonzero((col == '') | (col == 'Unknown'))
    print(f'{name}: Unknown/rỗng = {num_unknown}')


Attrition_Flag: Unknown/rỗng = 0
Gender: Unknown/rỗng = 0
Education_Level: Unknown/rỗng = 1519
Marital_Status: Unknown/rỗng = 749
Income_Category: Unknown/rỗng = 1112
Card_Category: Unknown/rỗng = 0


In [4]:
# phát hiện outlier bằng IQR cho một vài cột số
cols_check_outlier = ['Total_Trans_Amt', 'Total_Trans_Ct', 'Credit_Limit']
for name in cols_check_outlier:
    if name not in numeric_cols:
        continue
    mask_out, lower, upper = detect_outliers_iqr(data[name])
    print(f'{name}: outlier = {np.count_nonzero(mask_out)}, khoảng [{lower:.2f}, {upper:.2f}]')

# data_no_outlier = remove_outliers_iqr(data, cols_check_outlier, factor=1.5)
# print('Kích thước sau khi bỏ outlier:', data_no_outlier.shape)

Total_Trans_Amt: outlier = 896, khoảng [-1722.75, 8619.25]
Total_Trans_Ct: outlier = 2, khoảng [-9.00, 135.00]
Credit_Limit: outlier = 984, khoảng [-10213.75, 23836.25]


In [5]:
# Chuẩn hóa Min-Max cho một vài cột (ví dụ dùng cho mô hình dựa trên khoảng [0,1])
minmax_cols = ['Customer_Age', 'Total_Trans_Amt', 'Total_Trans_Ct']
minmax_data = {}
for name in minmax_cols:
    if name not in numeric_cols:
        continue
    minmax_data[name] = min_max_normalize(data[name])

print('Ví dụ 5 giá trị sau Min-Max cho Total_Trans_Amt:')
print(minmax_data['Total_Trans_Amt'][:5])


Ví dụ 5 giá trị sau Min-Max cho Total_Trans_Amt:
[0.03527317 0.04345165 0.07661066 0.03677534 0.01702459]


In [6]:
# Log-transform cho cột có phân phối lệch phải mạnh (ví dụ: Total_Trans_Amt)
log_cols = ['Total_Trans_Amt', 'Credit_Limit']
log_data = {}
for name in log_cols:
    if name not in numeric_cols:
        continue
    log_data[name] = log_transform(data[name])

print('Ví dụ 5 giá trị sau log-transform cho Total_Trans_Amt:')
print(log_data['Total_Trans_Amt'][:5])

Ví dụ 5 giá trị sau log-transform cho Total_Trans_Amt:
[7.04228617 7.16317239 7.54274355 7.06561336 6.70441436]


In [7]:
# Decimal scaling cho một cột (ví dụ: Credit_Limit)
scaled_credit = decimal_scaling(data['Credit_Limit'])
print('5 giá trị Credit_Limit sau decimal scaling:')
print(scaled_credit[:5])

5 giá trị Credit_Limit sau decimal scaling:
[0.12691 0.08256 0.03418 0.03313 0.04716]


In [8]:
# Chuẩn hóa z-score cho tất cả cột số (phục vụ các mô hình dựa trên gradient)
X_num = np.zeros((len(data), len(numeric_cols)))
for i, name in enumerate(numeric_cols):
    X_num[:, i] = standardize_zscore(data[name])

print('Kích thước ma trận numeric sau z-score:', X_num.shape)
print('Trung bình gần đúng của từng cột (mong đợi ~0):')
print(np.round(X_num.mean(axis=0), 3))
print('Phương sai gần đúng của từng cột (mong đợi ~1):')
print(np.round(X_num.std(axis=0), 3))

Kích thước ma trận numeric sau z-score: (10127, 14)
Trung bình gần đúng của từng cột (mong đợi ~0):
[ 0.  0. -0.  0. -0.  0.  0. -0. -0.  0. -0.  0. -0. -0.]
Phương sai gần đúng của từng cột (mong đợi ~1):
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [9]:
# Giữ nguyên các cột phân loại
# Chuẩn hóa z-score cho tất cả cột số
processed_dir = os.path.join(repo_root, 'data/processed')
os.makedirs(processed_dir, exist_ok=True)

# Quy ước: đưa cột phân loại trước, sau đó đến cột số
cols_order = list(cat_cols) + list(numeric_cols)

n_rows = len(data)
n_cols = len(cols_order)
processed_str = np.empty((n_rows, n_cols), dtype=object)

for j, name in enumerate(cols_order):
    col = data[name]
    if name in numeric_cols:
        col_proc = standardize_zscore(col)  # z-score
        processed_str[:, j] = col_proc.astype(str)
    else:
        processed_str[:, j] = col.astype(str)

header = ",".join(cols_order)
out_path = os.path.join(processed_dir, 'processed_data.csv')
np.savetxt(out_path, processed_str, fmt='%s', delimiter=',', header=header, comments='')

print('Đã lưu file tiền xử lý tại:', out_path)


Đã lưu file tiền xử lý tại: d:\Đại học\Kỳ 7\Lớp - Programming for DS\Lab\Lab 3 - Numpy\Program-for-Data-Science\data/processed\processed_data.csv


In [10]:
# Encode biến mục tiêu và lọc đa cộng tuyến cho biến số
# Attrition_Flag: Existing Customer -> 0, Attrited Customer -> 1
y = np.array([1 if v == 'Attrited Customer' else 0 for v in data['Attrition_Flag']], dtype=int)

# Cột phân loại dùng cho feature (bỏ target gốc dạng string)
cat_feature_cols = [c for c in cat_cols if c != 'Attrition_Flag']

# Lọc đa cộng tuyến đơn giản: loại các cột có |corr| > 0.9
num_matrix = np.vstack([data[name].astype(float) for name in numeric_cols]).T
corr_mat = np.corrcoef(num_matrix, rowvar=False)

keep_numeric = []
drop_numeric = set()
for i, name_i in enumerate(numeric_cols):
    if name_i in drop_numeric:
        continue
    keep_numeric.append(name_i)
    for j in range(i + 1, len(numeric_cols)):
        name_j = numeric_cols[j]
        if name_j in drop_numeric:
            continue
        if abs(corr_mat[i, j]) > 0.9:
            drop_numeric.add(name_j)

numeric_cols_filtered = keep_numeric
print('Cột số ban đầu:', numeric_cols)
print('Cột số sau khi lọc đa cộng tuyến:', numeric_cols_filtered)
print('Các cột số bị loại:', drop_numeric)


Cột số ban đầu: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
Cột số sau khi lọc đa cộng tuyến: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
Các cột số bị loại: {'Avg_Open_To_Buy'}


In [11]:
# Feature Engineering: Ordinal Encoding và One-Hot Encoding
n = len(data)
feature_list = []
feature_names = []

def add_feature(col_values, name):
    feature_list.append(col_values.astype(float))
    feature_names.append(name)

# 1. Thêm các biến số (đã lọc đa cộng tuyến)
for name in numeric_cols_filtered:
    add_feature(data[name].astype(float), name)

# 2. Ordinal Encoding
edu_order = {
    'Uneducated': 0,
    'High School': 1,
    'College': 2,
    'Graduate': 3,
    'Post-Graduate': 4,
    'Doctorate': 5,
    'Unknown': 0,  # coi Unknown như mức thấp nhất
}
edu_ord = np.array([edu_order.get(v, 0) for v in data['Education_Level']], dtype=float)
add_feature(edu_ord, 'Education_Level_ord')

income_order = {
    'Less than $40K': 1,
    '$40K - $60K': 2,
    '$60K - $80K': 3,
    '$80K - $120K': 4,
    '$120K +': 5,
    'Unknown': 0,  # giữ Unknown như 1 nhóm riêng
}
income_ord = np.array([income_order.get(v, 0) for v in data['Income_Category']], dtype=float)
add_feature(income_ord, 'Income_Category_ord')

card_order = {
    'Blue': 0,
    'Silver': 1,
    'Gold': 2,
    'Platinum': 3,
}
card_ord = np.array([card_order.get(v, 0) for v in data['Card_Category']], dtype=float)
add_feature(card_ord, 'Card_Category_ord')

# 3. One-Hot Encoding cho Marital_Status và Gender
for col in ['Marital_Status', 'Gender']:
    values = np.unique(data[col])
    for v in values:
        new_name = f'{col}_{v}'
        one_hot = (data[col] == v).astype(float)
        add_feature(one_hot, new_name)

# Kết hợp thành ma trận đặc trưng X
X = np.vstack(feature_list).T
print('Kích thước X (chưa scale):', X.shape)
print('Số feature:', len(feature_names))


Kích thước X (chưa scale): (10127, 22)
Số feature: 22


In [12]:
# Chia tập train/test (80/20)
rng = np.random.default_rng(42)
indices = np.arange(len(y))
rng.shuffle(indices)

split_idx = int(0.8 * len(y))
train_idx = indices[:split_idx]
test_idx = indices[split_idx:]

X_train = X[train_idx]
X_test = X[test_idx]
y_train = y[train_idx]
y_test = y[test_idx]

print('Train size:', X_train.shape, 'Test size:', X_test.shape)


Train size: (8101, 22) Test size: (2026, 22)


In [13]:
# Feature Scaling: Standardization (z-score) cho tất cả feature trừ one-hot (0/1)
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

for i, name in enumerate(feature_names):
    # one-hot bắt đầu bằng 'Marital_Status_' hoặc 'Gender_'
    if name.startswith('Marital_Status_') or name.startswith('Gender_'):
        continue
    col_train = X_train[:, i]
    mean = col_train.mean()
    std = col_train.std()
    if std == 0:
        X_train_scaled[:, i] = 0.0
        X_test_scaled[:, i] = 0.0
    else:
        X_train_scaled[:, i] = (X_train[:, i] - mean) / std
        X_test_scaled[:, i] = (X_test[:, i] - mean) / std

print(X_train_scaled[0])


[ 0.08184209  1.26615584  0.00917077 -0.52817756 -1.329695   -1.31590656
 -0.77326672 -1.42257922  0.12896056  0.3814442   0.68907275  0.36411967
 -0.99445898 -0.50991591 -0.05650204 -0.24707776  0.          0.
  1.          0.          1.          0.        ]


In [14]:
# Handling Imbalance: Oversampling đơn giản lớp thiểu số trên tập train
class_counts = np.bincount(y_train)
minority_class = np.argmin(class_counts)
majority_class = np.argmax(class_counts)

n_min = class_counts[minority_class]
n_maj = class_counts[majority_class]
print('Trước oversample:', class_counts)

if n_min < n_maj:
    diff = n_maj - n_min
    idx_min = np.where(y_train == minority_class)[0]
    rng = np.random.default_rng(42)
    sampled_idx = rng.choice(idx_min, size=diff, replace=True)

    X_add = X_train_scaled[sampled_idx]
    y_add = y_train[sampled_idx]

    X_train_bal = np.vstack([X_train_scaled, X_add])
    y_train_bal = np.concatenate([y_train, y_add])
else:
    X_train_bal = X_train_scaled.copy()
    y_train_bal = y_train.copy()

print('Sau oversample:', np.bincount(y_train_bal))


Trước oversample: [6794 1307]
Sau oversample: [6794 6794]


In [15]:
# Lưu train/test sau Feature Engineering + Scaling + Oversampling
processed_dir = os.path.join(repo_root, 'data', 'processed')
os.makedirs(processed_dir, exist_ok=True)

header = ','.join(feature_names + ['Attrition_Flag'])

train_mat = np.concatenate([X_train_bal, y_train_bal.reshape(-1, 1)], axis=1)
test_mat = np.concatenate([X_test_scaled, y_test.reshape(-1, 1)], axis=1)

train_path = os.path.join(processed_dir, 'train.csv')
test_path = os.path.join(processed_dir, 'test.csv')

np.savetxt(train_path, train_mat, delimiter=',', fmt='%.6f', header=header, comments='')
np.savetxt(test_path, test_mat, delimiter=',', fmt='%.6f', header=header, comments='')

print('Đã lưu train.csv tại:', train_path)
print('Đã lưu test.csv tại:', test_path)


Đã lưu train.csv tại: d:\Đại học\Kỳ 7\Lớp - Programming for DS\Lab\Lab 3 - Numpy\Program-for-Data-Science\data\processed\train.csv
Đã lưu test.csv tại: d:\Đại học\Kỳ 7\Lớp - Programming for DS\Lab\Lab 3 - Numpy\Program-for-Data-Science\data\processed\test.csv
